In [8]:
# https://arxiv.org/pdf/1801.09573.pdf
# Datasets
# We have collected the 1000 images from the internet in 2 categories as muffin and Chihuahua
# including 200 images from Oxford pet animal dataset [7] as shown in Figure 3. Please note here, all
# resources as images from the internet is for research purpose only, we don’t own any of them. ImageNet
# [5] also includes 1750 Chihuahua and 1335 various type of muffin images already.

# http://image-net.org/synset?wnid=n02085620 : 1750 Chihuahua
# http://image-net.org/synset?wnid=n07690273 : 1335 muffins

In [9]:
# https://aviaryan.com/blog/gsoc/downloading-files-from-urls
# import requests
# def is_downloadable(url):
#     """
#     Does the url contain a downloadable resource
#     """
#     try:
#         h = requests.head(url, allow_redirects=True)
#     except:
#         return False
#     header = h.headers
#     content_type = header.get('content-type')
#     if content_type == None:
#         return False
#     if 'text' in content_type.lower():
#         return False
#     if 'html' in content_type.lower():
#         return False
#     return True

# file1 = open('chihuahua.txt', 'r')
# Lines = file1.readlines()
# count = 0
# for line in Lines:
#     print(line)
#     if(is_downloadable(line)):
#         count += 1
#         r = requests.get(line, allow_redirects=True)
#         open(f'data/chihuahua/{count}.jpg', 'wb').write(r.content)

In [10]:
# file1 = open('muffin.txt', 'r')
# Lines = file1.readlines()
# count = 0
# for line in Lines:
#     print(line)
#     if(is_downloadable(line)):
#         count += 1
#         r = requests.get(line, allow_redirects=True)
#         open(f'data/muffin/{count}.jpg', 'wb').write(r.content)

In [11]:
import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
import time
import os
from copy import copy
from copy import deepcopy
import torch.nn.functional as F

# Allow augmentation transform for training set, no augementation for val/test set
# Normalize(mean, std, inplace=False)
# mean, std = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)
mean, std = (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)

preprocess_augment = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)])

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)])


full_train_dataset = torchvision.datasets.ImageFolder('data/train')
print(full_train_dataset)
train_dataset, val_dataset = torch.utils.data.random_split(full_train_dataset, [1600, 1988-1600])
train_dataset.dataset = copy(full_train_dataset)
train_dataset.dataset.transform = preprocess_augment
val_dataset.dataset.transform = preprocess

# DataLoaders for the three datasets
BATCH_SIZE=4
NUM_WORKERS=2
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE,shuffle=True , num_workers=NUM_WORKERS)
val_dataloader   = torch.utils.data.DataLoader(val_dataset  , batch_size=BATCH_SIZE,shuffle=False, num_workers=NUM_WORKERS)



Dataset ImageFolder
    Number of datapoints: 1988
    Root location: data/train


In [12]:
# import matplotlib.pyplot as plt
# import numpy as np
# # get some random training images
# dataiter = iter(train_dataloader)
# images, labels = dataiter.next()
# print(labels)
# toshow = torchvision.utils.make_grid(images)
# toshow = toshow / 2 + 0.5     # unnormalize
# npimg = toshow.numpy()
# plt.imshow(np.transpose(npimg, (1, 2, 0)))
# plt.show()
# classes = np.array(['chihuahua','muffin'])
# print(labels, classes[labels])

In [13]:
dataloaders = {'train': train_dataloader, 'val': val_dataloader}

from myNetwork.myResNet import ResNet
from trainer import trainer

def SEResNet18(num_classes = 10):
    return ResNet(ResNet._BLOCK_SEBASIC, [2, 2, 2, 2], num_classes)

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)
model = SEResNet18()
model.load_state_dict(torch.load('result-20210129-104338/seresnet18_adam_0.01.pth'))
model.classifier[2] = nn.Linear(512,2)
model.eval()


cuda:1


ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv2): Sequential(
    (0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (1): Sequential(
      (0): SEBasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (se): SELayer(
          (avg_pool): AdaptiveAvgPool2d(output_size=1)
          (fc): Sequential(
            (0): Linear(in_features=64, out_features=4, bias=False)
            (1): ReLU(inplace=True)
            (2): Linear(in_feat

In [14]:
model.to(device)
criterion = nn.CrossEntropyLoss()
params_to_update = model.parameters()
# Now we'll use Adam optimization
optimizer = optim.Adam(params_to_update, lr=0.01)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode='min',patience=5)
t = trainer(device,criterion, optimizer,scheduler)
model = t.train(model, dataloaders, num_epochs=60, weights_name='seresnet18_chihuahua_muffin_adam_0.01')

Epoch 0/59:LR: 0.01
----------
train Loss: 0.3794 Acc: 0.8294
Epoch time taken:  7.2649407386779785
val Loss: 0.2332 Acc: 0.8918
Epoch time taken:  8.395422458648682
Epoch 1/59:LR: 0.01
----------
train Loss: 0.2488 Acc: 0.9025
Epoch time taken:  6.999926567077637
val Loss: 0.1990 Acc: 0.9149
Epoch time taken:  8.068832635879517
Epoch 2/59:LR: 0.01
----------
train Loss: 0.1951 Acc: 0.9269
Epoch time taken:  7.136178731918335
val Loss: 0.2252 Acc: 0.9021
Epoch time taken:  8.171086311340332
Epoch 3/59:LR: 0.01
----------
train Loss: 0.1778 Acc: 0.9294
Epoch time taken:  7.13777756690979
val Loss: 0.1986 Acc: 0.9021
Epoch time taken:  8.165367126464844
Epoch 4/59:LR: 0.01
----------
train Loss: 0.1650 Acc: 0.9350
Epoch time taken:  7.021339416503906
val Loss: 0.1912 Acc: 0.8995
Epoch time taken:  8.055789232254028
Epoch 5/59:LR: 0.01
----------
train Loss: 0.1542 Acc: 0.9431
Epoch time taken:  7.03818678855896
val Loss: 0.1969 Acc: 0.9201
Epoch time taken:  8.069774627685547
Epoch 6/59:

In [19]:
test_dataset = torchvision.datasets.ImageFolder('data/test', transform=preprocess)
print(test_dataset)

Dataset ImageFolder
    Number of datapoints: 16
    Root location: data/test

In [15]:
t.test(model,test_dataloader)

NameError: name 'test_dataloader' is not defined